In [1]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearnex import patch_sklearn, config_context
patch_sklearn()
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import plotly.express as px


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
past = 'DataSet/Final/DataSet2017_2021.csv'
df = pd.read_csv(past, delimiter = ';',  low_memory=False, encoding='UTF-8', index_col = False)
df.columns

Index(['data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio',
       'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia',
       'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via',
       'uso_solo', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves',
       'ilesos', 'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude',
       'regional', 'delegacia', 'uop', 'ano', 'mes', 'dia', 'mortes'],
      dtype='object')

In [3]:
df1 = df.copy()
#df1 = df1[df['uf'] == 'MG']
#df2 = df[['uf','causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via','uso_solo', 'latitude', 'longitude']]
#df1['uf_br'] = df1['uf'] + ' ' + df1['br'].map(str)
col1 = ['uf', 'longitude', 'latitude', 'mes','causa_acidente', 'tipo_pista', 'tracado_via', 'mortos', 'condicao_metereologica'] # colunas selecionadas
col2 = ['uf', 'mes', 'tipo_pista', 'tracado_via', 'condicao_metereologica', 'causa_acidente'] # colunas categorizada
col3 = ['tipo_pista', 'mes', 'tracado_via', 'condicao_metereologica', 'causa_acidente', 'mortos'] #atributos de estudo

ci = 2
cf = 100

df2 = df1.copy()
df2 = df2[col1]
df2 = df2.reset_index(drop=True)
df3 = df2.copy()
df3

,uf,longitude,latitude,mes,causa_acidente,tipo_pista,tracado_via,mortos,condicao_metereologica
0,PR,-52.387894,-23.098807,January,Fenomenos Da Natureza,Simples,Reta,0,Chuva
1,SC,-48.635700,-27.810100,January,Falta De Atencao A Conducao,Dupla,Curva,0,Chuva
2,PR,309.935131,-23.369520,January,Animais Na Pista,Simples,Reta,0,Garoa/Chuvisco
3,GO,-48.969090,-16.274737,January,Avarias E/Ou Desgaste Excessivo No Pneu,Dupla,Reta,0,Ceu Claro
4,SC,-49.201670,-26.446752,January,Ingestao De Alcool,Simples,Nao Informado,0,Chuva
...,...,...,...,...,...,...,...,...,...
303768,ES,-40.876781,-20.856895,November,Velocidade Incompativel,Simples,Curva,0,Ceu Claro
303769,ES,-40.617537,-20.657276,May,Condutor Deixou De Manter Distancia Do Veiculo...,Simples,Reta,0,Ceu Claro
303770,SC,-48.631008,-27.383995,May,Velocidade Incompativel,Multipla,Nao Informado,1,Nublado
303771,MG,-48.211853,-18.986906,March,Reacao Tardia Ou Ineficiente Do Condutor,Dupla,Reta,0,Ceu Claro


In [4]:
le = preprocessing.LabelEncoder()
for i in col2:
    df3[i] = le.fit_transform(df3[i])

In [5]:
#df3[col2] = stats.zscore(df3[col2])
#df3 = StandardScaler().fit_transform(df3)
#df3 = pd.DataFrame(df3, columns = df3.columns)
#df3 = StandardScaler().fit_transform(df3)
#df3 = pd.DataFrame(df3, columns = df1.columns)*
kk = col1
df3[kk] = (df3[kk] - df3[kk].min()) / ( df3[kk].max() - df3[kk].min())
df3

,uf,longitude,latitude,mes,causa_acidente,tipo_pista,tracado_via,mortos,condicao_metereologica
0,0.653846,1.0,1.0,0.363636,0.500000,1.0,0.555556,0.000000,0.111111
1,0.884615,1.0,1.0,0.363636,0.452381,0.0,0.000000,0.000000,0.111111
2,0.653846,1.0,1.0,0.363636,0.095238,1.0,0.555556,0.000000,0.222222
3,0.307692,1.0,1.0,0.363636,0.142857,0.0,0.555556,0.000000,0.000000
4,0.884615,1.0,1.0,0.363636,0.547619,1.0,0.333333,0.000000,0.111111
...,...,...,...,...,...,...,...,...,...
303768,0.269231,1.0,1.0,0.818182,1.000000,1.0,0.000000,0.000000,0.000000
303769,0.269231,1.0,1.0,0.727273,0.178571,1.0,0.555556,0.000000,0.000000
303770,0.884615,1.0,1.0,0.727273,1.000000,0.5,0.333333,0.047619,0.777778
303771,0.384615,1.0,1.0,0.636364,0.821429,0.0,0.555556,0.000000,0.000000


In [ ]:
cluster = range(ci,cf)

metodo = ['k-means++', 'random']
CHS = []

for j in metodo:
    for i in cluster:
    
        model = KMeans(n_clusters = i, init=j, max_iter = 1000)
        model.fit(df3)
        CHS.append([j,metrics.calinski_harabasz_score(df3, model.labels_), i])
print('terminou')

In [ ]:
CHS2 = pd.DataFrame(CHS, columns = ['meth', 'chs', 'cluster'])

In [ ]:
fig = px.line(CHS2, x = 'cluster', y = 'chs', color = 'meth', title="Unsorted Input")
fig.update_traces(textposition="bottom right")
fig.update_layout(
    title="Plot Title",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title",
    legend_title="Legend Title",
)
fig.show()

In [ ]:
m = CHS2['chs'].max()
met = CHS2[CHS2['chs'] == m].meth.values[0]
n = CHS2[CHS2['chs'] == m].cluster.values[0]
model = KMeans(n_clusters = n, init=met, max_iter = 1000)
model.fit(df3)
dfresult = df1.copy()
dfresult = dfresult[col1]
dfresult['cluster'] = model.labels_
dfresult

print(met, str(n))

In [ ]:
dcenter = dfresult.groupby(['cluster']).size().reset_index(name='Frequency')
dcenter
for i,a in enumerate(col1):
    dcenter[a] = model.cluster_centers_[:,i]
dcenter

In [ ]:
dre = []
for j in range(0,n):
    print('Cluster ' + str(j))
    dd = dfresult[dfresult['cluster'] == j]
    for i in col3:
        d = dd[[i]].copy()
        dfselect = d.groupby([i]).size().reset_index(name='Frequency')
        dfselect = dfselect.sort_values(by=['Frequency'], ascending=False)
        dfselect['Frequencia %'] = (dfselect['Frequency']/dfselect['Frequency'].sum())*100
        dre.append(dfselect)
        print(dfselect)

In [ ]:
dfuf = dfresult.groupby(['cluster', 'uf']).size().reset_index(name='Frequency')
for i in range(0,n):
    fig = px.pie(dfuf[dfuf['cluster']==i], names ='uf', values = 'Frequency', title='Population of American continent')
    fig.update_traces(textposition='inside', textinfo='percent+label')
    #fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    fig.show()